In [3]:

import requests

import matplotlib.pyplot as plt
import seaborn as sns

import umap, numpy as np, pandas as pd


2025-07-02 22:56:08.597520: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 22:56:08.607428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751489768.618844 4138196 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751489768.622065 4138196 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751489768.630867 4138196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Or for specific warning types:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [5]:
labyrinthus_df = pd.read_parquet("../data/labyrinthus_grela_v0-1-2.parquet")
labyrinthus_df.sample(5)


,author,title,grela_id,sentence_id,sentence_text,context_3sents,tokens,concordance_tokens,not_before,not_after,date_random,lagt_genre,lagt_provenience,noscemus_genre,noscemus_discipline,target_token_id,target_char_start,target_char_end
850,"Melanchthon, Philipp",Initia doctrinae physicae dictata in academia ...,noscemus_906123,noscemus_906123_4158,"Hanc seu ueriorem, seu minus obscuram Alberti ...","Et primas partes oportet esse dissimiles, alio...","[{'char_end': 4, 'char_start': 0, 'lemma': 'hi...","[{'char_end': 4, 'char_start': 0, 'lemma': 'hi...",1549.0,1549.0,1549.0,None,None,Textbook,"Physics, Astronomy/Astrology/Cosmography, Mete...",62325724,69,80
801,"Gessner, Conrad",Clarissimi medici et philosophi eximii Conradi...,noscemus_830009,noscemus_830009_32025,ferrique per foramen latius maiorem eius porti...,"Postea uero, quam omnia productionum a cerebro...","[{'char_end': 5, 'char_start': 0, 'lemma': 'fe...","[{'char_end': 289, 'char_start': 281, 'lemma':...",1562.0,1562.0,1562.0,None,None,"Biography, Bibliography",Medicine,60401079,347,358
469,"Riccioli, Giovanni Battista",Almagestum Novum astronomiam veterem novamque ...,noscemus_699869,noscemus_699869_1081,Etenim Deus ipse sic omnia haec in Numero Pond...,in dextra uero Argum centuria oculorum non con...,"[{'char_end': 6, 'char_start': 0, 'lemma': 'et...","[{'char_end': 71, 'char_start': 70, 'lemma': '...",1651.0,1665.0,1653.0,None,None,Encyclopedic work,Astronomy/Astrology/Cosmography,16363288,123,133
643,"Vadianus, Joachim",Pomponii Melae Hispani libri de situ orbis tre...,noscemus_735140,noscemus_735140_5881,in quo de Labyrinthis.,"de quibus nemo Plinio uberius retulit, capite ...","[{'char_end': 2, 'char_start': 0, 'lemma': 'in...","[{'char_end': 38, 'char_start': 37, 'lemma': '...",1518.0,1518.0,1518.0,None,None,"Commentary, Encyclopedic work","Astronomy/Astrology/Cosmography, Geography/Car...",70906439,10,21
611,"Kircher, Athanasius","Mundus subterraneus, in XII libros digestus, q...",noscemus_732034,noscemus_732034_3948,"siquidem tium labyrinthos abductus, aut piscib...","quos praecipitem, sed nunquam amplius compaeni...","[{'char_end': 8, 'char_start': 0, 'lemma': 'si...","[{'char_end': 111, 'char_start': 105, 'lemma':...",1665.0,1665.0,1665.0,None,None,Encyclopedic work,"Geography/Cartography, Meteorology/Earth scien...",131481178,14,25


In [171]:
SYSTEM = """
You are a Latin philologist and semantic analyst.

You will classify how the Latin word “labyrinthus” is used in context.

Return exactly one label in the following format:
    [overall-context]-[detailed-sub-context]

Examples of valid overall contexts:
    mythological, medical, scientific, religious, metaphorical

Examples of valid sub-contexts:
    architectural, anatomical, alchemical, astronomical, spiritual, poetic, error, natural

The output must be a single dash-separated pair, lowercase only.
Do not output anything else — no explanations, comments, or punctuation.
""".strip()


FEWSHOT = """
Passage:
Labyrinthum erroris intexat.
Label: metaphorical-error

Passage:
In quo quicunque notitiam aliquam Trinitatis ... labyrinthum diuinae reuelationis ingreditur.
Label: religious-spiritual

Passage:
Unde praemonendum puto ... ne quis irrationabili aestimet voluptate ... labyrinthum erroris intexat.
Label: metaphorical-error

Passage:
inter corticem et lignum, Habitat in truncis Pini, ubi labyrinthos format ...
Label: scientific-natural

Passage:
Jauditoria Arteria Interna ... in labyrinthi cauitatem intrat ...
Label: medical-anatomical

Passage:
{0}
Label:
""".strip()

def classify_labyrinthus(passage):
    # Handle NaN or None values
    if pd.isna(passage) or passage is None:
        return "unknown"

    # Ensure passage is a string
    passage = str(passage)

    url = "http://localhost:11434/api/generate"
    prompt = FEWSHOT.format(passage.strip())
    payload = {
        "model": "llama3.1:70b",
        "system": SYSTEM,
        "prompt": prompt,
        "options": {
            "temperature": 0,
            "num_predict": 5
        },
        "stream": False
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()["response"].strip().lower()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


classify_labyrinthus(labyrinthus_df.sample()["context_3sents"].iloc[0])

'architectural-myth'

In [172]:
%%time
from concurrent.futures import ThreadPoolExecutor, as_completed

def classify_many(passages, max_workers=30):
    results = [None]*len(passages)
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        fut_to_i = {ex.submit(classify_labyrinthus, p): i
                    for i, p in enumerate(passages)}
        for fut in as_completed(fut_to_i):
            i = fut_to_i[fut]
            results[i] = fut.result()
    return results

labyrinthus_df["classification"] = classify_many(labyrinthus_df["context_3sents"])


CPU times: user 3.02 s, sys: 1.55 s, total: 4.57 s
Wall time: 1h 20min 18s


In [174]:
labyrinthus_df["classification"].value_counts()

classification
mythological-arch           166
medical-anatomical          154
metaphorical-error          143
metaphorical-poetic         102
scientific-anatomical        77
scientific-astronomical      36
architectural-myth           32
metaphorical-spirit          21
scientific-natural           18
architectural-arch           17
medical-error                14
scientific-architectural     13
architectural-poetic         11
scientific-alchemical        11
religious-spiritual           9
architectural                 7
metaphorical-arch             7
architectural-natural         7
metaphorical-alchemical       6
alchemical-anatomical         6
architectural-anatom          5
scientific-error              5
medical-alchemical            4
mythological-poetic           3
geographical-architect        3
scientific-metaphorical       3
medical-metaphorical          3
historical-poetic             2
metaphorical-phil             2
philosophical-error           2
alchemical-poetic        

In [180]:
import re

# ------------------------------------------------------------------
# 1. canonical heads
HEADS = {
    "mythological", "medical", "metaphorical", "scientific",
    "religious", "alchemical", "poetic", "geographical", "philosophical",
    "architectural"   # include because sometimes it *is* the head
}

# 2. abbreviation expansion  (regex pattern -> full form)
ABBR_MAP = {
    r"\barch\b"       : "architectural",
    r"\barchitect\b"  : "architectural",
    r"\banatom\b"     : "anatomical",
    r"\balchem\b"     : "alchemical",
    r"\bmetaph?\b"    : "metaphorical",
    r"\bmyth\b"       : "mythological",
    r"\bphil\b"       : "philosophical",
    r"\bspirit\b"     : "spiritual",
    r"\bnat\b"        : "natural",
}

ABBR_REPLACERS = [(re.compile(pat), rep) for pat, rep in ABBR_MAP.items()]

# 3. second-part synonym merge  (maps rare variant -> canonical)
SECOND_PART_MERGE = {
    "architect": "architectural",
    "monument" : "architectural",
    "acoustic" : "architectural",
    "error"    : "error",          # keeps as is
    "spirit"   : "spiritual",
    "met"      : "metaphorical",
    "nat"      : "natural",
}

# ------------------------------------------------------------------
def expand(segment: str) -> str:
    for rgx, rep in ABBR_REPLACERS:
        segment = rgx.sub(rep, segment)
    segment = SECOND_PART_MERGE.get(segment, segment)
    return segment

def normalize_label(label: str) -> str:
    parts = label.strip().lower().split("-")
    if not parts:
        return label

    # expand abbreviations / synonyms
    parts = [expand(p) for p in parts]

    # if one part duplicates (architectural-architectural) collapse
    if len(parts) == 2 and parts[0] == parts[1]:
        return parts[0]

    # ensure first part is a canonical head; if not but second is, swap
    if len(parts) >= 2:
        if parts[0] not in HEADS and parts[1] in HEADS:
            parts[0], parts[1] = parts[1], parts[0]

    return "-".join(parts[:2])   # keep only first two segments for simplicity

In [201]:
labyrinthus_df["classification"] = labyrinthus_df["classification"].apply(normalize_label)

In [202]:
labyrinthus_df["classification"].value_counts()

classification
mythological-architectural     198
medical-anatomical             154
metaphorical-error             143
metaphorical-poetic            102
scientific-anatomical           77
scientific-astronomical         36
metaphorical-spiritual          21
scientific-natural              18
architectural-architectural     17
medical-error                   14
scientific-architectural        13
poetic-architectural            13
scientific-alchemical           11
religious-spiritual              9
metaphorical-architectural       8
architectural                    7
architectural-natural            7
metaphorical-alchemical          6
alchemical-anatomical            6
architectural-anatomical         5
scientific-error                 5
medical-alchemical               4
medical-metaphorical             3
geographical-architectural       3
mythological-poetic              3
scientific-metaphorical          3
metaphorical-philosophical       2
poetic-historical                2
alche

In [ ]:
labyrinthus_df.to_parquet("../data/labyrinthus_df_classified.parquet")